In [ ]:
#%pip install anthropic

In [ ]:
from dotenv import dotenv_values,load_dotenv
import sys
sys.path.append('../common')
from helper import Helper
import anthropic
load_dotenv()

secrets = dotenv_values("secrets.env")
api_key = secrets.get("API_KEY")
environment = secrets.get("APIM_NAME")
charge_code = "12345"
region_override = secrets.get("REGION_OVERRIDE")

helper = Helper(environment, 'genai/anthropic', api_key, region_override)

client = anthropic.Anthropic(
    base_url=F"https://{environment}/genai/anthropic",
    api_key=api_key,
    default_headers=
    {
        "Ocp-Apim-Subscription-Key": api_key,
        "x-kpmg-charge-code": charge_code,
        "x-kpmg-region-override": region_override,
    }
)

In [ ]:
full_response = []
helper.log("Anthropic Streaming messages test")

with client.messages.stream(
    model="claude-3-5-sonnet-20241022",
    messages = [
        {"role": "user", "content":"Hello"}
    ],
    max_tokens=200
) as stream:
    for text in stream.text_stream:
        full_response.append(text)
        helper.log(text, end="")


helper.log("\n", end="")
assert len(full_response) > 0

### Anthropic Non streaming messages test

In [ ]:
helper.log("Anthropic Non streaming messages test")

response = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=200,
    messages = [
        {"role": "user", "content":"Hello"}
    ]
)

assert len(response.content) > 0

helper.log(response.content[0].text + "\n")

### Anthropic Array type in the message request test

In [ ]:
helper.log("Anthropic Array type in the message request test")

response = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=200,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/png",
                        "data": "iVBORw0KGgoAAAANSUhEUgAAAQAAAAEACAIAAADTED8xAAADMElEQVR4nOzVwQnAIBQFQYXff81RUkQCOyDj1YOPnbXWPmeTRef+/3O/OyBjzh3CD95BfqICMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMK0CMO0TAAD//2Anhf4QtqobAAAAAElFTkSuQmCC",
                    },
                },
                {
                    "type": "text",
                    "text": "Describe this image."
                }
            ],
        }
    ],
)

assert len(response.content) > 0
helper.log(response.content[0].text + "\n")

#### Anthropic Token count (Beta) test

In [ ]:
helper.log("Anthropic Count tokens test")

response = client.beta.messages.count_tokens(
    model="claude-3-5-sonnet-20241022",
            messages = [
        {"role": "user", "content":"Hello"}
    ]
)

assert response.input_tokens > 0

helper.log(F'Input Tokens: {response.input_tokens}' + "\n")

#### Anthropic Negative tests

In [ ]:
helper.log("Anthropic Negative test: Check if the model is not found in user configuration file")

try:
    response = client.messages.create(
        model="non-claude-3-5-sonnet-20241022",
        max_tokens=200,
        messages=[
            {"role": "user", "content": "Hello"}
        ]
    )

    assert False, "Expected Error but got response: " + response.content[0].text + "\n"

except Exception as e:
    if 'model was not found in the user configuration' in str(e):
        helper.log("Expected Error: Model was not found in user configuration file.")
        assert True
    else :
        helper.log("Unexpected Error: " + str(e) + "\n")
        assert False



In [ ]:
helper.log("Anthropic Negative test: Check if the charge code is not passed, an error is returned")

client = anthropic.Anthropic(
    base_url=F"https://{environment}/genai/anthropic",
    api_key=api_key,
    default_headers=
    {
        "Ocp-Apim-Subscription-Key": api_key,
        "x-kpmg-region-override": region_override,
    }
)

try:
    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=200,
        messages=[
            {"role": "user", "content": "Hello"}
        ]
    )

    assert False, "Expected Error but got response: " + response.content[0].text + "\n"

except Exception as e:
    if 'The HTTP header is missing the required \'x-kpmg-charge-code\' header' in str(e):
        helper.log("Expected Error: Missing charge code header.")
        assert True
    else :
        helper.log("Unexpected Error: " + str(e) + "\n")
        assert False